<a href="https://colab.research.google.com/github/PauloHFS/descriptive-data-analysis-2023-1/blob/main/Lab5_Intervalo_Confianca.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab - Intervalo de Confiança

* A partir o dataset IRIS (já trabalhado na aula), gere um novo dataset contendo apenas 1 das 4 medidas disponíveis no dataset original (Sepal Lentgh, Setal Width, Petal Length, Petal Width). Além disso, remova todas as linhas referentes a um dos tipos de flor e mantenha apenas 2 tipos ao invés dos 3 contidos no datset original (Setosa, Versicolour e Virginica). Ou seja, seu dataset conterá 1 coluna e 100 linhas (há 50 amostras de cada tipo de flor no datset original).

* Usando bootstrap para gerar o intervalo de confiança, calcule a diferença entre as médias para os dois tipos de flor.

* Informe o seu resultado em uma frase do tipo: "Com 95% de confiança, podemos afirmar que, em média, os cachorros são mais altos do que os gatos em 20,3cm, com intervalo de confiança de 17,2cm a 23,5cm".

* **Use o dataset inteiro** considerando que ele representa toda a amostra que você conseguiu coletar. Observe que isso é diferente do que fizemos em aula, onde consideramos que as 150 flores eram todas as existentes e nossa amostra continha uma quantidade menor (20, 30, por exemplo).

* Dependendo da forma como você pretenda resolver o problema, você pode ter interesse em alterar a função bootstrap disponibilizada abaixo. Fique à vontade para gerar variações dessa função se assim desejar.



## Dados do(a) aluno(a)

**Nome**: Paulo Hernane Fontes e Silva

**Matrícula**: 119210311

## Imports.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample

## Carregando o Datset Original

In [3]:
from sklearn import datasets
iris = datasets.load_iris()

In [4]:
iris.data

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
       [4.9, 3

In [5]:
iris.feature_names

['sepal length (cm)',
 'sepal width (cm)',
 'petal length (cm)',
 'petal width (cm)']

In [6]:
iris.target

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [7]:
iris.target_names

array(['setosa', 'versicolor', 'virginica'], dtype='<U10')

## Gerando o novo Dataset

In [8]:
# vamos pegar a setosa e a versicolor e comparar o length das petalas
data_setosa, data_versicolor = iris.data[iris.target == 0, 2], iris.data[iris.target == 1, 2]

In [9]:
data_setosa.shape

(50,)

## Definição da função de Bootstrap

Dependendo da forma como você pretenda resolver o problema, você pode ter interesse em alterar a função bootstrap disponibilizada abaixo. Fique à vontade para gerar variações dessa função se assim desejar.

In [10]:
def bootstrap(data, func, alpha=0.95, n_rep=1000):

  theta_hat_error_list = list()
  theta = func(data)

  # Resampling
  for i in range(0,n_rep):
    x = resample(data, replace=True)
    theta_hat_error = func(x) - theta
    theta_hat_error_list.append(theta_hat_error)

  # Percentiles
  lower_p = ((1.0-alpha)/2.0) * 100
  upper_p = (alpha+((1.0-alpha)/2.0)) * 100

  # Lower and Upper Bounds
  lower_value = np.percentile(theta_hat_error_list, lower_p)
  upper_value = np.percentile(theta_hat_error_list, upper_p)

  lower_limit = theta+lower_value
  upper_limit = theta+upper_value

  return theta, lower_limit, upper_limit

## Análise das diferenças entre os dois tipos de flor escolhidos.

Primeiro realizamos uma permutação calculando a diferença de todos os elementos da nossa amostragem para obter a média das diferenças geral, iremos utilizar para avaliar nossa amostragem.

In [23]:
# Realizando broadcasting para calcular a diferença dos 50 X 50 elementos
# cada elemento de data_versicolor vira uma lista com esse mesmo elemento,
# em seguida o np calcula a diferença desse elemento por todos os elemento em
# data_setosa e armazena nessa lista
diff = data_versicolor[:, np.newaxis] - data_setosa
# reduzimos as dimensões das array agora para apenas uma
diff = diff.flatten()
diff.shape

(2500,)

In [93]:
def bootstrap_new(data1, data2, theta, alpha=0.95, n_rep=1000):
 # 1. pegar um valor estatistico da amostra. aka média das diferenças aka theta
  theta_hat_error_list = list()

  # Resampling
  for i in range(0,n_rep):
    # 2. realiza reamostragem de ambos os datasets de amostra
    x,y = resample(data1,data2, replace=True)

    # 3. calcula a diferença das médias entre as amostragens
    # 4. calcula o error dessa diferença de theta
    # obs: aqui testei fazer a diferença por broadcasting das series mas deu muito proximo o resultado, então deixei esse mesmo
    theta_hat_error = (np.mean(x) - np.mean(y)) - theta
    # 5. adiciona a lista
    theta_hat_error_list.append(theta_hat_error)

  # Percentiles
  lower_p = ((1.0-alpha)/2.0) * 100
  upper_p = (alpha+((1.0-alpha)/2.0)) * 100

  # Lower and Upper Bounds
  lower_value = np.percentile(theta_hat_error_list, lower_p)
  upper_value = np.percentile(theta_hat_error_list, upper_p)

  lower_limit = theta+lower_value
  upper_limit = theta+upper_value

  return theta, lower_limit, upper_limit

In [95]:
bootstrap_new(data_versicolor, data_setosa, np.mean(diff))

(2.798, 2.652000000000001, 2.938099999999999)

Resultado:

Podemos afirmar com 95% de certeza que as Versicolor são maiores que as Setosas em 2.798cm, com intervalo de confiança de 2.649cm a 2.934cm.